In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Function

In [2]:
def an_filename(name):
    '''Get the real file name not the path from csv file'''
    filename = "a_" + name.split("a_")[1]
    return filename

def sen_filename(name):
    '''Get the real file name not the path from csv file'''
    filename = "s_" + name.split("s_")[1]
    return filename

In [3]:
def convert(seconds): 
    milli = (seconds % 1) * 100
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
      
    return "%d:%02d:%02d:%02d" % (hour, minutes, seconds, milli)

In [4]:
def key(X):
    key, value = list(X.items())[0]
    return key.strip("()")

def value(X):
    key, value = list(X.items())[0]
    key2, value2 = list(value[0].items())[0]
    return value2

def process(X):
    key, value = list(X.items())[0]
    key2, value2 = list(value[0].items())[0]
    return key2

### Prepare Annotation dataframe

In [5]:
anno_basepath = "../raw_data/full_data/" # using for iterate path
anno_csvpath = "../raw_data/sample/annotations.csv"
an_csv = pd.read_csv(anno_csvpath)
an_csv["real_filename"] = an_csv["filename"].apply(an_filename)
an_csv # make real file name for iterate path

,time,filename,real_filename
0,2020-08-05 22:21:03,data/raw/2020-08-05-asa/a_2020-08-05-W6Ex5DZKn...,a_2020-08-05-W6Ex5DZKnuMe8eiFIsbSutgVLV72.json


In [6]:
an_df = pd.read_json("../raw_data/sample/annot/" + an_csv["real_filename"][0])

In [7]:
an_df["activities"] = an_df["dt"].apply(key)
an_df["process"] = an_df["dt"].apply(process)
an_df["time"] = an_df["dt"].apply(value)
an_df = an_df[an_df.columns[-3:]]

In [8]:
row = an_df.shape[0]
n = 0
list_begin = []
list_end = []
list_act = []
for row in range(int(row/2)):
    list_begin.append(an_df["time"][n])
    list_act.append(an_df["activities"][n])
    list_end.append(an_df["time"][n+1])
    n +=2

In [9]:
an_dict_clean = {"activitiy" : list_act, "start" : list_begin, "end" : list_end}
an_df_clean = pd.DataFrame(an_dict_clean)

In [10]:
an_df_clean.head()

,activitiy,start,end
0,down_stairs,1596618985598,1596618989833
1,down_stairs,1596618995255,1596618997833
2,down_stairs,1596619066247,1596619069306
3,down_stairs,1596619073407,1596619075856
4,down_stairs,1596619079973,1596619082796


In [11]:
batch = []
batch_no = 0
start = []
end = []
activitiy = []
for row in range(an_df_clean.shape[0]):
    start_an = an_df_clean.loc[row, "start"]
    end_an = start_an + 2000
    if end_an > an_df_clean.loc[row, "end"]:
        continue
    start.append(start_an)
    end.append(end_an)
    activitiy.append(an_df_clean.loc[row, "activitiy"])
    batch.append(batch_no)
    batch_no += 1
    start_an += 2000

In [12]:
an_dict_clean = {"activitiy" : activitiy, "start" : start, "end" : end, "batch" : batch}
an_df_clean = pd.DataFrame(an_dict_clean)

In [14]:
an_df_clean.head()

,activitiy,start,end,batch
0,down_stairs,1596618985598,1596618987598,0
1,down_stairs,1596618995255,1596618997255,1
2,down_stairs,1596619066247,1596619068247,2
3,down_stairs,1596619073407,1596619075407,3
4,down_stairs,1596619079973,1596619081973,4


### Prepare Sensor dataframe

In [15]:
sensor_csvpath = "../raw_data/sample/sensors.csv"
sen_csv = pd.read_csv(sensor_csvpath)
sen_csv["real_filename"] = sen_csv["filename"].apply(sen_filename)
sen_csv.head()

,time,filename,real_filename
0,2020-08-05 22:21:01,data/raw/2020-08-05-asa/s_2020-08-05-159661889...,s_2020-08-05-1596618899641-ASUS_X013DB-W6Ex5DZ...
1,2020-08-05 22:21:01,data/raw/2020-08-05-asa/s_2020-08-05-159661890...,s_2020-08-05-1596618902316-SHV40-W6Ex5DZKnuMe8...
2,2020-08-05 22:21:01,data/raw/2020-08-05-asa/s_2020-08-05-159661890...,s_2020-08-05-1596618909689-ASUS_X013DB-W6Ex5DZ...
3,2020-08-05 22:21:01,data/raw/2020-08-05-asa/s_2020-08-05-159661891...,s_2020-08-05-1596618912434-SHV40-W6Ex5DZKnuMe8...
4,2020-08-05 22:21:01,data/raw/2020-08-05-asa/s_2020-08-05-159661891...,s_2020-08-05-1596618919758-ASUS_X013DB-W6Ex5DZ...


In [16]:
sen_path_base = "../raw_data/sample/sens/"

In [17]:
sample_path = sen_path_base + sen_csv["real_filename"][50];sample_path

'../raw_data/sample/sens/s_2020-08-05-1596619151947-ASUS_X013DB-W6Ex5DZKnuMe8eiFIsbSutgVLV72.json'

In [18]:
sen_json = pd.read_json(sample_path)
sen_json.head()

,dt,os,pm,p,uid,v
a,"[[1596619141863.0, 1077989232403.0, 0.72761535...",22,ASUS_X013DB,0,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1
g,[],22,ASUS_X013DB,0,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1
p,[],22,ASUS_X013DB,0,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1


In [19]:
sen_acc = np.array(sen_json.dt["a"])
acc_df = pd.DataFrame(sen_acc, columns=["time_stamp", "sensor_time_stamp","X","Y","Z"])

In [20]:
def check_between(time_stamp):
    for i in range(an_df_clean.shape[0]):
        if an_df_clean.loc[i,"start"] < time_stamp and time_stamp < an_df_clean.loc[i,"end"]:
            return an_df_clean.loc[i,"activitiy"] + " " + str(an_df_clean.loc[i,"batch"])   
    return None

In [21]:
#All file in the folder
all_jason_df = pd.DataFrame(columns=["time_stamp","sensor_time_stamp","X","Y","Z"])
for sen_jsonpath in sen_csv["real_filename"]:
    sensor_path = sen_path_base + sen_jsonpath
    sen_json = pd.read_json(sensor_path)
    if sen_json.dt["a"] == []:
        continue
    sen_acc = np.array(sen_json.dt["a"])
    acc_df = pd.DataFrame(sen_acc, columns=["time_stamp", "sensor_time_stamp","X","Y","Z"])
    all_jason_df = pd.concat([all_jason_df,acc_df])

In [22]:
all_jason_df["activity"] = all_jason_df["time_stamp"].apply(check_between)

In [24]:
all_jason_df.isnull().sum()

time_stamp                0
sensor_time_stamp         0
X                         0
Y                         0
Z                         0
activity             240089
dtype: int64

In [25]:
all_jason_df.drop("sensor_time_stamp",axis = 1,inplace=True)
all_jason_df.to_csv("sample.csv", index=False)

### Fall data

In [ ]:
fall_path = "../raw_data/20201130-sg9-y-fall/a_2020-11-30-W6Ex5DZKnuMe8eiFIsbSutgVLV72.json"
fall_an = pd.read_json(fall_path)

In [ ]:
fall_df

In [ ]:
fall_an["activities"] = fall_an["dt"].apply(key)
fall_an["process"] = fall_an["dt"].apply(process)
fall_an["time"] = fall_an["dt"].apply(value)
fall_an = fall_an[fall_an.columns[-3:]]

In [ ]:
row = fall_an.shape[0]
n = 0
list_begin = []
list_end = []
list_act = []
for row in range(int(row/2)):
    list_begin.append(fall_an["time"][n])
    list_act.append(fall_an["activities"][n])
    list_end.append(fall_an["time"][n+1])
    n +=2

In [ ]:
fall_clean = {"activitiy" : list_act, "start" : list_begin, "end" : list_end}
fall_df = pd.DataFrame(fall_clean)

In [ ]:
fall_df["diff_time"] = (fall_df["end"] - fall_df["start"])//2000 

Fall data is different we are not using 2s bin otherwise it going to get dropped ! OwO